In [34]:
from feature.clip_make_dataset import make_data
from feature.after_dataset import accumulate
import pandas as pd
from glob import glob
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import glob
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression, Ridge
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [18]:
train_input_list = './data/train_input/*.csv'
train_target_list = './data/train_target/*.csv'

test_input_list = './data/test_input/*.csv'
test_target_list = './data/test_target/*.csv'

In [19]:
train, test, p_train, p_test = make_data(train_input_list, train_target_list, test_input_list, test_target_list)

In [20]:
train_rol = pd.read_csv('./data/train_roling.csv')
test_rol = pd.read_csv('./data/test_roling.csv')
train_rol = train_rol.drop(['DAT'], axis=1)
test_rol = test_rol.drop(['DAT'], axis=1)

In [21]:
accumulate(train, test, '누적분무량')
accumulate(train, test, '누적백색광량')
accumulate(train, test, '누적청색광량')
accumulate(train, test, '누적적색광량')

In [22]:
# train = pd.concat([train, train_1, train_3, train_std], axis=1)
# test = pd.concat([test, test_1, test_3, test_std], axis=1)

# train = train.drop(['index'], axis=1)
# test = test.drop(['index'], axis=1)
train = train.reset_index()
test  = test.reset_index()
train = pd.concat([train, train_rol], axis=1)
test = pd.concat([test, test_rol], axis=1)

# train = pd.concat([train, p_train], axis=1)
# test = pd.concat([test, p_test], axis=1)

In [29]:
def model_train(train, test, model):
    '''
    train shape = (784, 18), test shape = (140, 18)
    model => sklearn api model
    '''

    X = train.drop(['predicted_weight_g', 'Case', 'obs_time', '6time'], axis=1)
    y = train['predicted_weight_g']

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=113, shuffle=True)
    
    x_test = test.drop(['predicted_weight_g', 'Case', 'obs_time', '6time'], axis=1)
    
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)

    rmse = mean_squared_error(y_val, y_val_pred)**0.5
    print(f"validation rmse: {rmse}")
    
    x_train = train.drop(['predicted_weight_g', 'Case', 'obs_time', '6time'], axis=1)
    y_train = train['predicted_weight_g']
    
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    x_test['predicted_weight_g'] = y_pred
    submit = x_test[['DAT', 'predicted_weight_g']]
    # submit['DAT'] = submit['DAT']
    all_target_list = sorted(glob.glob('./data/test_target/*.csv'))
    for idx, test_path in enumerate(all_target_list):
        submit_df = pd.read_csv(test_path)
        submit_df['predicted_weight_g'] = submit['predicted_weight_g'][idx*28:idx*28+28].values
        submit_df.to_csv(test_path, index=False)

In [39]:
xgb = XGBRegressor()
cat = CatBoostRegressor(verbose=100, early_stopping_rounds=50, random_seed=113)
lgbm = LGBMRegressor()
linear = LinearRegression()
rig = Ridge(alpha=10)

model_train(train, test, cat)

Learning rate set to 0.037231
0:	learn: 40.8360278	total: 161ms	remaining: 2m 40s
100:	learn: 7.6906061	total: 1.65s	remaining: 14.7s
200:	learn: 3.7596402	total: 3.09s	remaining: 12.3s
300:	learn: 2.4499390	total: 5.25s	remaining: 12.2s
400:	learn: 1.7453132	total: 7.51s	remaining: 11.2s
500:	learn: 1.3277612	total: 8.9s	remaining: 8.87s
600:	learn: 1.0293528	total: 10.3s	remaining: 6.86s
700:	learn: 0.8360205	total: 11.8s	remaining: 5.02s
800:	learn: 0.6791757	total: 13.9s	remaining: 3.45s
900:	learn: 0.5584532	total: 16s	remaining: 1.76s
999:	learn: 0.4652840	total: 17.4s	remaining: 0us
validation rmse: 6.081672653847054
Learning rate set to 0.039399
0:	learn: 40.4997203	total: 17.6ms	remaining: 17.6s
100:	learn: 6.9459083	total: 1.55s	remaining: 13.8s
200:	learn: 3.6782770	total: 3.4s	remaining: 13.5s
300:	learn: 2.5742399	total: 6.02s	remaining: 14s
400:	learn: 1.8996634	total: 8s	remaining: 11.9s
500:	learn: 1.5057937	total: 9.62s	remaining: 9.58s
600:	learn: 1.1992054	total: 11.

In [43]:
X = train.drop(['predicted_weight_g', 'Case', 'obs_time', '6time', 'index'], axis=1)
y = train['predicted_weight_g']
from sklearn.model_selection import StratifiedKFold, KFold

skf = KFold(n_splits=5, shuffle=True, random_state=404)

folds = []

for train_idx, val_idx in skf.split(X, y):
    folds.append((train_idx, val_idx))

XGB_model = {}

for f in range(5):
    print(
        f'===================================={f+1}============================================')
    train_idx, val_idx = folds[f]

    x_train, x_val, y_train, y_val = X.iloc[train_idx], X.iloc[val_idx], y.iloc[train_idx], y.iloc[val_idx]

    XGB = CatBoostRegressor(random_seed=42, early_stopping_rounds=35, verbose=500)
    XGB.fit(x_train, y_train)

    y_pred = XGB.predict(x_val)
    rmse = mean_squared_error(y_val, y_pred)**0.5
    print(f"{f + 1} Fold RMSE = {rmse}")
    XGB_model[f] = XGB
    print(f'================================================================================\n\n')


submit = test[['DAT', 'predicted_weight_g']]
x_test = test.drop(['predicted_weight_g', 'Case', 'obs_time', '6time', 'index'], axis=1)
for fold in range(5):
    submit['predicted_weight_g'] += XGB_model[fold].predict(x_test)/5
all_target_list = sorted(glob.glob('./data/test_target/*.csv'))
for idx, test_path in enumerate(all_target_list):
    submit_df = pd.read_csv(test_path)
    submit_df['predicted_weight_g'] = submit['predicted_weight_g'][idx*28:idx*28+28].values
    submit_df.to_csv(test_path, index=False)

In [71]:
df_imp = pd.DataFrame({'imp':XGB.feature_importances_}, index = XGB.feature_names_)
df_imp = df_imp[df_imp.imp > 0].sort_values('imp').copy()
df_imp

,imp
"('시간당적색광량', 23)",0.000257
"('시간당총광량', 8)",0.001222
"('ec관측치', 9)",0.001261
"('시간당적색광량', 4)",0.001446
"('ec관측치', 11)",0.001637
...,...
"('co2관측치', 9)",1.364377
월간누적백색광량,2.667871
월간누적적색광량,4.488112
월간누적분무량,15.613039
